In [2]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [3]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='breast_cancer', task='classification', batch_size=32)

xtrain size 455
xtest size 114
y_train size 455
y_test size 114


In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [5]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp4b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [6]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        print(f'Trail {i}: \n')
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [ ]:
output_directory = './output.csv'
run_experiment(DTEmlp4b, trails=5, output_directory=output_directory)

In [8]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [12]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)

Randomly initializing 4 models
Training Model with id 1
in multi


Training Process:   0%|          | 0/10 [00:00<?, ?it/s]

Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.51it/s]


Total Epochs: 10
Train loss start: 1.1862322092056274, Train loss end: 1.1452547311782837
Highest Mean Train Accuracy (over epoch): 0.2455952380952381
Test loss start: 1.1184672117233276, Test loss end: 1.0856622457504272
Highest mean Test Accuracy (over epoch): 0.21831597222222224
 Test F1 score at highest accuracy: 0.34836603710719377
./saved_models/model_1.pth 

Training Model with id 2
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 18.98it/s]


...

Total Epochs: 10
Train loss start: 1.1078389883041382, Train loss end: 1.067595362663269
Highest Mean Train Accuracy (over epoch): 0.36190476190476184
Test loss start: 1.0804970264434814, Test loss end: 1.0767598152160645
Highest mean Test Accuracy (over epoch): 0.4338541666666667
 Test F1 score at highest accuracy: 0.35320665120238676
./saved_models/model_8.pth 

Training Model with id 6
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 16.18it/s]


Total Epochs: 10
Train loss start: 1.027978777885437, Train loss end: 0.8756952285766602
Highest Mean Train Accuracy (over epoch): 0.5307142857142857
Test loss start: 1.0475984811782837, Test loss end: 1.0313153266906738
Highest mean Test Accuracy (over epoch): 0.5672743055555556
 Test F1 score at highest accuracy: 0.5471617509805599
./saved_models/model_6.pth 

 Maximum accuracy: 0.6355034722222221, F1: 0.43374965989264247, Test: 1.0280076310038566, Train: 1.01976021528244
in multi


Training Process: 100%|██████████| 70/70 [00:05<00:00, 11.71it/s]

Total Epochs: 70
Train loss start: 1.1676043272018433, Train loss end: 0.7814117670059204
Highest Mean Train Accuracy (over epoch): 0.6431377551020409
Test loss start: 1.1446529626846313, Test loss end: 0.9602816104888916
Highest mean Test Accuracy (over epoch): 0.5249255952380952
 Test F1 score at highest accuracy: 0.7131814410490723
